Imports

In [13]:
import gurobipy as gp
from gurobipy import *
from gurobipy import GRB
import numpy as np
import random
import pandas as pd
import math
# import scipy.sparse as sp

In [14]:
# clases
from entities import Dia, Horario, BloqueHorario, Profesor, Materia, Grupo, Prioridad, Superposicion

In [15]:
# variables
from variables import u, v, w, x, y, z

# Functions

In [16]:
# cargar datos

# crear materia
def add_materia(materias, nombre, carga_horaria, cantidad_dias, grupos=[], profesores=[], cantidad_profesores=1):
    id = len(materias)
    materias.append(Materia(id, nombre, carga_horaria=carga_horaria, cantidad_dias=cantidad_dias,
                            grupos=grupos, profesores=profesores, cantidad_profesores=cantidad_profesores))

# crear grupo
def add_grupo(grupos, anio, turno, carrera, particion, recurse, aux):
    id = len(grupos)
    grupos.append(Grupo(id, anio, turno, carrera, particion, bool(recurse), aux))

# crear profesor
def add_profesor(profesores, nombre, minimizar_dias=False):
    id = len(profesores)
    profesor = Profesor(id, nombre, minimizar_dias)
    if not profesor in profesores:
        profesores.append(profesor)

def lista_profesores(profesores, nombres):
    profs = []
    for n in nombres:
        for p in profesores:
            if str(p) == n:
                profs.append(p)
                break
    return profs

def lista_grupos(grupos, nombres):
    gs = []
    for n in nombres:
        for g in grupos:
            if str(g) == n:
                gs.append(g)
                break
    return gs


In [17]:
#prioridades
def update_no_disp(profesor, bloques_horario, no_disp_index):
    for i in no_disp_index:
        if bloques_horario[i] not in profesor.no_disponible:
            profesor.no_disponible.append(bloques_horario[i])

def update_prioridad(profesor, bloques_horario, array_prioridad):
    # array_prioridad[i] = [(d,h),a]
    for i in array_prioridad:
        b_id = i[0]
        value = i[1]
    
        if value == 0:
            update_no_disp(profesor, bloques_horario, [b_id])
        
        prior = Prioridad(value, bloques_horario[b_id], profesor = profesor)
        if not (prior in profesor.prioridades):
            profesor.prioridades.append(prior)

def random_pr(bloques_horario):
    pr_array = []
    for b in bloques_horario:
        pr_array.append([b, random.randint(0,3)])

    return pr_array

def fixed_pr(bloques_horario, value):
    pr_array = []
    for b in bloques_horario:
        pr_array.append([b, value])
        
    return pr_array

def update_no_disp_mat(materia, bloques_horario, no_disp_index):
    for i in no_disp_index:
        if bloques_horario[i] not in materia.no_disponible:
            materia.no_disponible.append(bloques_horario[i])

def update_prioridad_mat(materia, bloques_horario, array_prioridad):
    # array_prioridad[i] = [(d,h),a]
    for i in array_prioridad:
        b_id = i[0]
        value = i[1]
    
        if value == 0:
            update_no_disp_mat(materia, bloques_horario, [b_id])
        
        prior = Prioridad(value, bloques_horario[b_id], materia = materia)
        if not (prior in materia.prioridades):
            materia.prioridades.append(prior)
    


In [18]:
#superposicion

def calcular_super(m1, m2):
    if m1 == m2:
        return Superposicion(0, m1, m2)
    else:
        s = False
        for g1 in m1.grupos:
            if g1 in m2.grupos:
                s = True
                break
        return Superposicion(1 if s else 0, m1, m2)

In [19]:
#imprimir
def print_timetable(dias, horarios, u_dict, w_dict, grupos, anios):
    for a in anios:
        print('\n', '-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -')
        print("Año: ", str(a))
        # filtrar grupos dentro del año
        for g in grupos_anio(grupos, a):
            print('\n', "Grupo: ", str(g))
            print('\t', *[str(d) for d in dias], sep='\t\t')
            print('··········································································································')
            for h in horarios_turno(horarios, g.turno):
                mats = [search_materia(BloqueHorario(d,h), g, u_dict) for d in dias]
                lista = []
                for m in mats:
                    if m is None:
                        lista.append("---")
                    else:
                        # mostrar profesor:
                        # ps = search_profesor(m, w_dict)
                        # lista.append(str(m) + " " + str([str(p) for p in ps]))
                        lista.append(str(m))
                print(str(h), *lista, sep='\t\t')

def print_prof_timetable(dias, horarios, u_dict, w_dict, profesores, materias):
    for p in profesores:
        if len(intersection(nombres(materias), p.materias())) == 0:
            continue
        print('\n', "Profesor: ", str(p))
        print('\t', *[str(d) for d in dias], sep='\t')
        print('·····················································')
        for h in horarios:
            mats = [search_materia_prof(BloqueHorario(d,h), p, u_dict, w_dict, materias) for d in dias]
            lista = []
            for m in mats:
                if m is None:
                    lista.append("---")
                else:
                    lista.append(str(m))
            print(str(h), *lista, sep='\t')

def intersection(array1, array2):
    ret = []
    for i in array1:
        if i in array2:
            ret.append(i)
    return ret

def nombres(array):
    return [i.nombre for i in array]

def print_prioridades(dias, horarios, profesores):
    for p in profesores:
        print('\n', "Profesor: ", str(p))
        print('\t', *[str(d) for d in dias], sep='\t')
        print('·····················································')
        for h in horarios:
            prioridades = [str(search_prioridad(BloqueHorario(d,h),p)) for d in dias]
            print(str(h), *prioridades, sep='\t')

def print_prioridades_materia(dias, horarios, materia):
    print("Materia: ", str(materia))    
    print_prioridades(dias, horarios, materia.profesores)

def search_materia(b, g, u_dict):
    for u_id in u_dict:
        u_obj = u_dict[u_id]
        if u_obj.horario == b and g in u_obj.materia.grupos and round(u_obj.variable.x) == 1:
            return u_obj.materia
    return None

def search_profesor(materia, w_dict):
    
    if materia is None:
        return []
    
    ps = []
    for p in materia.profesores:
        if round(w_dict[materia.id, p.id].variable.x) == 1:
            ps.append(p)
    return ps

def search_profesor_by_nombre(profesores, nombre):
    for p in profesores:
        if p.nombre == nombre:
            return p
    return None

def search_materias_by_nombre(materias, nombre):
    ret = []
    for m in materias:
        if m.nombre == nombre:
            ret.append(m)
    return ret
    
def search_materia_prof(b, p, u_dict, w_dict, materias):
    for m in materias:
        if round(w_dict[m.id, p.id].variable.x * u_dict[m.id, b.id()].variable.x) == 1:
            return m
    #     ps = search_profesor(u_obj.materia, w_dict)
    #     if bloque == b and p in ps and round(u_obj.variable.x) == 1:
    #         return u_obj.materia

    # for u_id in u_dict:
    #     u_obj = u_dict[u_id]
    #     bloque = u_obj.horario
    #     ps = search_profesor(u_obj.materia, w_dict)
    #     if bloque == b and p in ps and round(u_obj.variable.x) == 1:
    #         return u_obj.materia
    return None

def search_prioridad(b, p):
    if b in p.no_disponible:
        return "-"
    for pr in p.prioridades:
        if pr.bloque_horario == b:
            return pr.value
        
def grupos_anio(grupos, anio):
    gs = []
    for g in grupos:
        if g.anio == anio:
            gs.append(g)
    return gs

def horarios_turno(horarios, turno):
    hs = []
    for h in horarios:
        if turno in h.turnos:
            hs.append(h)
    return hs

def horarios_ids_turno(horarios, turno):
    hs = []
    for h in horarios:
        if turno in h.turnos:
            hs.append(h.id)
    return hs



In [20]:
# imprimir para mostrar en Excel
def print_timetable_excel(dias, horarios, u_dict, w_dict, grupos, anios):
    for a in anios:
        print('\n', '-  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -')
        print("Año: ", str(a))
        # filtrar grupos dentro del año
        for g in grupos_anio(grupos, a):
            print('\n', "Grupo: ", str(g))
            # print('\t', *[str(d) for d in dias], sep='\t\t')
            print('', *[str(d) for d in dias], sep='\t')
            for h in horarios_turno(horarios, g.turno):
                mats = [search_materia(BloqueHorario(d,h), g, u_dict) for d in dias]
                lista = []
                for m in mats:
                    if m is None:
                        lista.append("---")
                    else:
                        # mostrar profesor:
                        ps = search_profesor(m, w_dict)
                        lista.append(str(m) + " " + str([str(p) for p in ps]))
                        # lista.append(str(m))
                # print(str(h), *lista, sep='\t\t')
                print(str(h), *lista, sep='\t')

def print_prof_timetable_excel(dias, horarios, u_dict, w_dict, profesores, materias):
    for p in profesores:
        if len(intersection(nombres(materias), p.materias())) == 0:
            continue
        print('\n', "Profesor: ", str(p))
        print('', *[str(d) for d in dias], sep='\t')
        for h in horarios:
            mats = [search_materia_prof(BloqueHorario(d,h), p, u_dict, w_dict, materias) for d in dias]
            lista = []
            for m in mats:
                if m is None:
                    lista.append("---")
                else:
                    lista.append(str(m))
            print(str(h), *lista, sep='\t')

def print_timetable_salones(dias, horarios, niveles):
    print('\t', *[str(d) for d in dias], sep='\t')
    print('··········································································································')
    for h in horarios:
        lista = [str(niveles[d.id, h.id])+"%" for d in dias]
        print(str(h), *lista, sep='\t')




In [108]:
#copiar resultados de variables a hoja de excel

def copy_variables_excel(u_dict, w_dict):

    data_u = {
        'materia': [],
        'dia': [],
        'horario': [],
        'U': []
    }
    for u_i in u_dict:
        data_u['materia'].append(u_dict[u_i].materia.id)
        data_u['dia'].append(u_dict[u_i].horario.dia.id)
        data_u['horario'].append(u_dict[u_i].horario.horario.id)
        data_u['U'].append(round(u_dict[u_i].variable.x))

    data_w = {
        'materia': [],
        'profesor': [],
        'W': []
    }
    for w_i in w_dict:
        data_w['materia'].append(w_dict[w_i].materia.id)
        data_w['profesor'].append(w_dict[w_i].profesor.id)
        data_w['W'].append(round(w_dict[w_i].variable.x))

    with pd.ExcelWriter("output.xlsx") as writer:
        pd.DataFrame(data_u).to_excel(writer, sheet_name="u", index=False)
        pd.DataFrame(data_w).to_excel(writer, sheet_name="w", index=False)


In [22]:
#buscar materias

def materias_profesor(profesor, materias_total):
    mats = []

    for m in materias_total:
        if profesor in m.profesores:
            mats.append(m)

    return mats

def agrupar_materias(lista_materias):
    lista_nombres = {}

    for m in lista_materias:
        if str(m) not in lista_nombres:
            lista_nombres[str(m)] = [m]
        else:
            lista_nombres[str(m)].append(m)

    return lista_nombres

def materias_grupo(grupo, materias_total):
    materias = []

    for m in materias_total:
        if grupo is not None and grupo in m.grupos:
            materias.append(m)
            
    return materias

def materias_grupo_ids(grupo, materias_total):
    materias_ids = []

    for m in materias_grupo(grupo, materias_total):
        if grupo is not None and grupo in m.grupos:
            materias_ids.append(m.id)
            
    return materias_ids


# Formulacion

## (1) Constantes / Datos

### Horarios

In [23]:
dias = [
    Dia(1, "lun"),
    Dia(2, "mar"),
    Dia(3, "mie"),
    Dia(4, "jue"),
    Dia(5, "vie")
]
dias_ids = []
for d in dias:
    dias_ids.append(d.id)

horarios = [
    Horario(1, "8:00", "8:50", ["m"]),
    Horario(2, "8:50", "9:40", ["m"]),
    Horario(3, "9:50", "10:40", ["m"]),
    Horario(4, "10:40", "11:30", ["m"]),
    Horario(5, "11:40", "12:30", ["m"]),
    Horario(6, "12:30", "13:20", ["m"]),
    Horario(7, "14:00", "14:50", ["t"]),
    Horario(8, "14:50", "15:40", ["t"]),
    Horario(9, "15:50", "16:40", ["t"]),
    Horario(10, "16:40", "17:30", ["t"])
]

horarios_ids = []
for h in horarios:
    horarios_ids.append(h.id)

bloques_horario = {}
for dia in dias:
    for horario in horarios:
        bloques_horario[(dia.id, horario.id)] = BloqueHorario(dia, horario)

bloques_horario_ids = []
for b in bloques_horario:
    bloques_horario_ids.append(b)


### Profesores

In [24]:
profesores = []

df = pd.read_excel("input.xlsm", sheet_name="profesores")

for i in range(len(df)):
    nombre = df["Profesor"][i]
    # # grupos_max = df["Max grupos"][i]
    # if not isinstance(grupos_max, int):
    #     grupos_max = None
    minimizar_dias = isinstance(df["Minimizar dias"][i], str)
    # if str(cantidad_dias) == "nan":
    #     cantidad_dias = None
    # else:
    #     cantidad_dias = int(cantidad_dias)
        
    if isinstance(nombre, str):
        add_profesor(profesores, nombre, bool(minimizar_dias))

profesores_ids = []
for p in profesores:
    profesores_ids.append(p.id)



c:\Users\marzu\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


### Prioridades

In [25]:
# prioridades random

# for p in profesores:
#     update_prioridad(p, bloques_horario, random_pr(bloques_horario))

In [26]:
# prioridades fijas

# for p in profesores:
#     update_prioridad(p, bloques_horario, fixed_pr(bloques_horario, 1)) # prioridad = 1 para todos los casos
    

In [27]:
# prioridades desde excel: 1 profesor por hoja

for prof in profesores:
    
    try:
        df = pd.read_excel("input.xlsm", sheet_name=str(prof))
    except:
        update_prioridad(prof, bloques_horario, random_pr(bloques_horario))
        continue

    array_prioridad = []
    # array_prioridad[i] = [(d,h),a]

    h_id = []

    for heading in df:

        if heading == str(prof):
            for row in df[heading]:
                h_id.append(row)
        else:
            i = 0
            for row in df[heading]:
                array_prioridad.append([(heading, h_id[i]), row])
                i += 1

    # print(array_prioridad)
    
    update_prioridad(prof, bloques_horario, array_prioridad)


### Grupos

In [28]:
#grupos
grupos = []


df = pd.read_excel("input.xlsm", sheet_name="grupos")

for i in range(len(df)):
    
    args = []
    args.append(df["Año"][i])  # anio
    args.append(df["Turno"][i])     # turno
    args.append(df["Carrera"][i])   # carrera
    args.append(df["Particion"][i])  # particion
    args.append(isinstance(df["Recurse"][i], str)) # recurse
    args.append(df["Auxiliar"][i])

    for i in range(0, len(args)):
        
        if str(args[i]) == "nan":
            args[i] = None

        try:
            args[i] = int(args[i])
        except:
            pass      

    add_grupo(grupos, *args)

grupos_ids = []
for g in grupos:
    grupos_ids.append(g.id)


In [29]:
print([str(g) for g in grupos])

['11', '12', '13', '1REC1', '1REC2', '2CIV', '2IND', '2INF1', '2INF2', '2TEL', '3CIV', '3IND', '3TEL', '3INF1', '3INF2', 'dummy']


In [30]:
anios = []
for g in grupos:
    if not g.anio is None and not g.anio in anios:
            anios.append(g.anio)


### Materias 

In [31]:
#materias
materias = []

df = pd.read_excel("input.xlsm", sheet_name="materias")

for i in range(len(df)):
    nombre = df["Materia"][i]
    if not isinstance(nombre, str):
        continue
    carga_horaria = df["Carga Horaria"][i]
    cantidad_dias = df["Días por semana"][i]
    
    try:
        cantidad_profesores = int(df["Cantidad profesores"][i])
    except:
        cantidad_profesores = 1

    profs = []
    p = 1
    while True:
        try:
            new_p = df["Profesor " + str(p)][i]
            if isinstance(new_p, str):
                profs.append(new_p)
            p += 1
        except:
            break

    gs = []
    g = 1
    while True:
        try:
            new_g = df["Grupo " + str(g)][i]
            if isinstance(new_g, str):
                gs.append(new_g)
            g += 1
        except:
            break
    
    # gs = lista_grupos(grupos, [x.strip() for x in str(df["Grupos"][i]).split(',')])
    # profs = lista_profesores(profesores, [x.strip() for x in str(df["Profesores"][i]).split(',')])

    add_materia(materias, nombre, carga_horaria, cantidad_dias, lista_grupos(grupos, gs), lista_profesores(profesores, profs), cantidad_profesores)



c:\Users\marzu\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [32]:
materias_ids = []
# materias_dict = {}
for m in materias:
    materias_ids.append(m.id)
    # materias_dict[m.id] = m


In [33]:
# materias por profesor:
df = pd.read_excel("input.xlsm", sheet_name="profesores")

for i in range(len(df)):
    nombre = df["Profesor"][i]

    lista_mats = []
    m = 1
    while True:
        try:
            new_m = df["Materia " + str(m)][i]
            
            if isinstance(new_m, str):
                grupos_max = int(df["Grupos materia " + str(m)][i])
                lista_mats.append({"nombre_materia" : new_m, "grupos_max" : grupos_max})
            m += 1
        except:
            break
    
    prof = search_profesor_by_nombre(profesores, nombre)
    if not prof is None:
        prof.lista_materias = lista_mats


    # add_materia(materias, nombre, carga_horaria, cantidad_dias, lista_grupos(grupos, gs), lista_profesores(profesores, profs))



c:\Users\marzu\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [34]:
print(profesores[8].lista_materias)

[{'nombre_materia': 'Fisica', 'grupos_max': 1}]


### Superposicion

In [35]:
# superposicion
superposicion = {}

for m1 in materias:
    for m2 in materias:
        superposicion[(m1.id, m2.id)] = calcular_super(m1, m2)


In [36]:
# prioridades de materia

# prioridades desde excel: 1 profesor por hoja

for mat in materias:
    
    try:
        df = pd.read_excel("input.xlsm", sheet_name=str(mat))
    except:
        # update_prioridad_mat(mat, bloques_horario, random_pr(bloques_horario))
        continue

    array_prioridad = []
    # array_prioridad[i] = [(d,h),a]

    h_id = []

    for heading in df:

        if heading == str(mat):
            for row in df[heading]:
                h_id.append(row)
        else:
            i = 0
            for row in df[heading]:
                array_prioridad.append([(heading, h_id[i]), row])
                i += 1

    # print(array_prioridad)
    update_prioridad_mat(mat, bloques_horario, array_prioridad)
    update_prioridad()

In [37]:
print("Grupos:", len(grupos))
# print([str(i) for i in grupos])
print("Materias:", len(materias))
# print([str(i) for i in materias])
print("Profesores:", len(profesores))
# print([str(i) for i in profesores])

Grupos: 16
Materias: 80
Profesores: 55


## (2) Variables


$$ u_{mb} \in \{0,1\} $$

    m: materia
    b: bloque horario = [dh]: dia y hora

$$ v_{md} \in \{0,1\} $$

    m: materia
    d: dia

$$ w_{mp} \in \{0,1\} $$

    m: materia
    p: profesor


$$ x_{gb} \in \{0,1\} $$

    g: grupo
    b: bloque horario = [dh]: dia y hora

$$ y_{pb} \in \{0,1\} $$

    p: profesor
    b: bloque horario = [dh]: dia y hora

$$ z_{pd} \in \{0,1\} $$

    p: profesor
    d: dia

In [38]:
# variables

u_dict = {}
for m in materias:   
    for b_id in bloques_horario:
        u_dict[(m.id, b_id)] = u(m, bloques_horario[b_id])
print("u: ", len(u_dict))

# print([str(us) for us in u_dict])

v_dict = {}
for m in materias:
    for d in dias:
        v_dict[(m.id, d.id)] = v(m, d)
print("v: ", len(v_dict))

w_dict = {}
for m in materias:
    for p in profesores:
        w_dict[(m.id, p.id)] = w(m, p)
print("w: ", len(w_dict))

x_dict = {}
for g in grupos:
    for b_id in bloques_horario:
        x_dict[(g.id, b_id)] = x(g, bloques_horario[b_id])
print("x: ", len(x_dict))

y_dict = {}
for p in profesores:
    for b_id in bloques_horario:
        y_dict[(p.id, b_id)] = y(p, bloques_horario[b_id])
print("y: ", len(y_dict))

z_dict = {}
for p in profesores:
    for d in dias:
        z_dict[(p.id, d.id)] = z(p, d)
print("z: ", len(z_dict))

u:  4000
v:  400
w:  4400
x:  800
y:  2750
z:  275


In [39]:
print(len(materias))
print(len(bloques_horario))

80
50


In [40]:
# Create a new model
model = gp.Model("timetable")

Set parameter ServerTimeout to value 30
Set parameter TokenServer to value "10.4.0.1"


In [41]:
# Create variables

# u_vars = m.addMVar(shape=len(u_dict), vtype=GRB.BINARY, name="u") # variable matrix

for u_i in u_dict: # crear variables "u" a partir de u_dict
    u_dict[u_i].variable = model.addVar(vtype=GRB.BINARY, name=str(u_dict[u_i]))

# print([ui.variable for ui in u_dict])

for v_i in v_dict: # crear variables "v" a partir de v_dict
    v_dict[v_i].variable = model.addVar(vtype=GRB.BINARY, name=str(v_dict[v_i]))

# v_vars = np.array([v_i.variable for v_i in v_dict])

# for r_i in r_dict:
#     r_dict[r_i].variable = model.addVar(vtype=GRB.CONTINUOUS, name=str(r_dict[r_i]))

for w_i in w_dict: # crear variables "v" a partir de v_dict
    w_dict[w_i].variable = model.addVar(vtype=GRB.BINARY, name=str(w_dict[w_i]))


for x_i in x_dict: # crear variables "x" a partir de x_dict
    x_dict[x_i].variable = model.addVar(vtype=GRB.BINARY, name=str(x_dict[x_i]))

for y_i in y_dict: # crear variables "y" a partir de y_dict
    y_dict[y_i].variable = model.addVar(vtype=GRB.BINARY, name=str(y_dict[y_i]))

for z_i in z_dict: # crear variables "z" a partir de z_dict
    z_dict[z_i].variable = model.addVar(vtype=GRB.BINARY, name=str(z_dict[z_i]))


## (3) Restricciones

### (3.1) Materias

#### (3.1.1) superposicion (redundante con la definicion de la variable x)
$$ \sum_{b} \sum_{m, m'}{u_{mb} \times S_{m,m'} \times u_{m'b}} = 0 $$


In [42]:
model.addConstrs(gp.quicksum(u_dict[m1, b].variable * superposicion[(m1, m2)].value * u_dict[m2, b].variable
                             for m1 in materias_ids for m2 in materias_ids)
                == 0 for b in bloques_horario_ids)

## for g in grupos:
##     mats = materias_grupo(g, materias)
##     if len(mats) > 1:
##         # model.addConstrs(gp.quicksum(u_dict[m.id, b].variable for m in mats) <= 1 for b in bloques_horario_ids)
##         model.addConstrs(gp.quicksum(u_dict[m.id, b].variable for m in mats) <= 1 for b in bloques_horario_ids)


{(1, 1): <gurobi.QConstr Not Yet Added>,
 (1, 2): <gurobi.QConstr Not Yet Added>,
 (1, 3): <gurobi.QConstr Not Yet Added>,
 (1, 4): <gurobi.QConstr Not Yet Added>,
 (1, 5): <gurobi.QConstr Not Yet Added>,
 (1, 6): <gurobi.QConstr Not Yet Added>,
 (1, 7): <gurobi.QConstr Not Yet Added>,
 (1, 8): <gurobi.QConstr Not Yet Added>,
 (1, 9): <gurobi.QConstr Not Yet Added>,
 (1, 10): <gurobi.QConstr Not Yet Added>,
 (2, 1): <gurobi.QConstr Not Yet Added>,
 (2, 2): <gurobi.QConstr Not Yet Added>,
 (2, 3): <gurobi.QConstr Not Yet Added>,
 (2, 4): <gurobi.QConstr Not Yet Added>,
 (2, 5): <gurobi.QConstr Not Yet Added>,
 (2, 6): <gurobi.QConstr Not Yet Added>,
 (2, 7): <gurobi.QConstr Not Yet Added>,
 (2, 8): <gurobi.QConstr Not Yet Added>,
 (2, 9): <gurobi.QConstr Not Yet Added>,
 (2, 10): <gurobi.QConstr Not Yet Added>,
 (3, 1): <gurobi.QConstr Not Yet Added>,
 (3, 2): <gurobi.QConstr Not Yet Added>,
 (3, 3): <gurobi.QConstr Not Yet Added>,
 (3, 4): <gurobi.QConstr Not Yet Added>,
 (3, 5): <guro

#### (3.1.2) cubrir carga horaria para cada materia
$$ \sum_b{u_{mb}} = C_m $$

    para todo m (c_m: carga horaria)

In [43]:
for m in materias:
    model.addConstr(gp.quicksum(u_dict[m.id, b].variable for b in bloques_horario_ids) == m.carga_horaria)

#### (3.1.3) particion de horas por materia

##### (3.1.3.1) fijar cantidad de dias por materia

$$ \sum_d {v_{md}} = D_m $$

    para todo m

In [44]:
for m in materias:
    model.addConstr(gp.quicksum(v_dict[m.id, d].variable for d in dias_ids) == m.cantidad_dias)

##### (3.1.3.2) fijar maximo y minimo de horas por dia

$$ v_{md} \times {H_{MIN}}_m \leq \sum_h {u_{mdh}} \leq v_{md} \times {H_{MAX}}_m $$

    para todo m, d

In [45]:
for m in materias:

    model.addConstrs(gp.quicksum(u_dict[(m.id,(d,h))].variable for h in horarios_ids)
                     <= m.horas_max() * v_dict[m.id,d].variable for d in dias_ids)
    
    model.addConstrs(gp.quicksum(u_dict[(m.id,(d,h))].variable for h in horarios_ids)
                     >= m.horas_min() * v_dict[m.id,d].variable for d in dias_ids)
    
    


##### (3.1.3.3) fijar materia a turno de horarios

$$ u_{mb} = 0 $$

    para todo b fuera de m.turnos
    para todo m



In [46]:
for m in materias:
    # no_bloques_materia_ids = [i for i in bloques_horario if not bloques_horario[i].horario.turno in m.turnos()]
    no_bloques_materia_ids = []
    for b in bloques_horario:
        if len([t for t in bloques_horario[b].horario.turnos if t in m.turnos()]) == 0:
            no_bloques_materia_ids.append(b)
    model.addConstrs(u_dict[m.id, b].variable == 0 for b in no_bloques_materia_ids)


#### (3.1.4) horas consecutivas dentro de un dia

$$ \sum_h {u_{mdh}} - \sum_h {u_{md(h)} · u_{md(h+1)}} = v_{md} $$
    para todo m, d

In [47]:
model.addConstrs(gp.quicksum(u_dict[(m,(d,h))].variable for h in horarios_ids)
                 - gp.quicksum(u_dict[(m,(d,h))].variable * u_dict[(m,(d,h+1))].variable for h in horarios_ids[0:-1])
                 == v_dict[m, d].variable for m in materias_ids for d in dias_ids)

{(0, 1): <gurobi.QConstr Not Yet Added>,
 (0, 2): <gurobi.QConstr Not Yet Added>,
 (0, 3): <gurobi.QConstr Not Yet Added>,
 (0, 4): <gurobi.QConstr Not Yet Added>,
 (0, 5): <gurobi.QConstr Not Yet Added>,
 (1, 1): <gurobi.QConstr Not Yet Added>,
 (1, 2): <gurobi.QConstr Not Yet Added>,
 (1, 3): <gurobi.QConstr Not Yet Added>,
 (1, 4): <gurobi.QConstr Not Yet Added>,
 (1, 5): <gurobi.QConstr Not Yet Added>,
 (2, 1): <gurobi.QConstr Not Yet Added>,
 (2, 2): <gurobi.QConstr Not Yet Added>,
 (2, 3): <gurobi.QConstr Not Yet Added>,
 (2, 4): <gurobi.QConstr Not Yet Added>,
 (2, 5): <gurobi.QConstr Not Yet Added>,
 (3, 1): <gurobi.QConstr Not Yet Added>,
 (3, 2): <gurobi.QConstr Not Yet Added>,
 (3, 3): <gurobi.QConstr Not Yet Added>,
 (3, 4): <gurobi.QConstr Not Yet Added>,
 (3, 5): <gurobi.QConstr Not Yet Added>,
 (4, 1): <gurobi.QConstr Not Yet Added>,
 (4, 2): <gurobi.QConstr Not Yet Added>,
 (4, 3): <gurobi.QConstr Not Yet Added>,
 (4, 4): <gurobi.QConstr Not Yet Added>,
 (4, 5): <gurobi

#### (3.1.5) evitar dias consecutivos para una misma materia

$$ \sum_m \sum_d {v_{m(d)} · v_{m(d+1)}} = 0 $$


In [48]:
model.addConstrs(gp.quicksum(v_dict[m,d].variable * v_dict[m,d+1].variable for d in dias_ids[0:-1]) == 0
                 for m in materias_ids)


{0: <gurobi.QConstr Not Yet Added>,
 1: <gurobi.QConstr Not Yet Added>,
 2: <gurobi.QConstr Not Yet Added>,
 3: <gurobi.QConstr Not Yet Added>,
 4: <gurobi.QConstr Not Yet Added>,
 5: <gurobi.QConstr Not Yet Added>,
 6: <gurobi.QConstr Not Yet Added>,
 7: <gurobi.QConstr Not Yet Added>,
 8: <gurobi.QConstr Not Yet Added>,
 9: <gurobi.QConstr Not Yet Added>,
 10: <gurobi.QConstr Not Yet Added>,
 11: <gurobi.QConstr Not Yet Added>,
 12: <gurobi.QConstr Not Yet Added>,
 13: <gurobi.QConstr Not Yet Added>,
 14: <gurobi.QConstr Not Yet Added>,
 15: <gurobi.QConstr Not Yet Added>,
 16: <gurobi.QConstr Not Yet Added>,
 17: <gurobi.QConstr Not Yet Added>,
 18: <gurobi.QConstr Not Yet Added>,
 19: <gurobi.QConstr Not Yet Added>,
 20: <gurobi.QConstr Not Yet Added>,
 21: <gurobi.QConstr Not Yet Added>,
 22: <gurobi.QConstr Not Yet Added>,
 23: <gurobi.QConstr Not Yet Added>,
 24: <gurobi.QConstr Not Yet Added>,
 25: <gurobi.QConstr Not Yet Added>,
 26: <gurobi.QConstr Not Yet Added>,
 27: <gurob

#### (3.1.6) indisponibilidad de materia

$$ u_{mb} = 0 $$

    para todo m sin disponibilidad en b

In [49]:
for m in materias:
    for b in m.no_disponible:
        model.addConstr(u_dict[m.id, b.id()].variable == 0)

### (3.2) Profesores

#### (3.2.1) indisponibilidad

$$ \sum_m w_{mp} \times u_{mb} = 0 $$

    para todo p sin disponibilidad en b

In [50]:
for m in materias:
    for p in m.profesores:
        for b in p.no_disponible:
            model.addConstr(u_dict[m.id, b.id()].variable * w_dict[m.id, p.id].variable == 0)

LINEALIZAR: SUSTITUIR CON VARIABLES y_pb

$$ y_{pb} = 0 $$

    para todo p sin disponibilidad en b

In [51]:
for p in profesores:
    for b in p.no_disponible:
        model.addConstr(y_dict[p.id, b.id()].variable == 0)

#### (3.2.2) unica materia por profesor para un mismo bloque horario

$$ \sum_{m} {w_{mp} \times u_{mb}} \leq 1 $$

    para todo b, p

NO ES NECESARIA (SE ASEGURA CON LA DEFINICION DE y_pb)

In [52]:
for p in profesores:
    model.addConstrs(gp.quicksum(u_dict[m, b].variable * w_dict[m, p.id].variable for m in materias_ids)
                     <= 1 for b in bloques_horario_ids)

#### (3.2.3) profesores por materia


##### (3.2.3.1) limitar profesores a lista

$$ w_{mp} = 0 $$

    para todo p not in m.lista_profesores
    para todo m

In [53]:
for m in materias:
    for p in profesores:
        if p not in m.profesores:
            model.addConstr(w_dict[m.id, p.id].variable == 0)

##### (3.2.3.2) cantidad de profesores para una misma materia

$$ \sum_p {w_{mp}} = P_m $$

    para todo m

In [54]:
for m in materias:
    model.addConstr(gp.quicksum(w_dict[m.id, p].variable for p in profesores_ids) == m.cantidad_profesores)

#### (3.2.4) carga horaria por docente: limitar la cantidad de materias por profesor


$$ \sum_m {w_{mp}} \leq K_p $$

    para todo p



In [55]:
# separar por nombre de materia
for p in profesores:
    for l in p.lista_materias:
        mats = search_materias_by_nombre(materias, l["nombre_materia"])
        model.addConstr(gp.quicksum(w_dict[m.id, p.id].variable for m in mats) <= l["grupos_max"])
        

#### (3.2.5) definicion de variable "y" (asignacion horario-profesor)

$$ y_{pb} = \sum_m{u_{mb}·w_{mp}} $$

    para todo p, b

In [56]:
model.addConstrs(y_dict[p, b].variable ==
                 gp.quicksum(u_dict[m,b].variable * w_dict[m,p].variable for m in materias_ids)
                 for b in bloques_horario_ids for p in profesores_ids)

{((1, 1), 0): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 1): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 2): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 3): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 4): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 5): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 6): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 7): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 8): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 9): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 10): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 11): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 12): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 13): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 14): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 15): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 16): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 17): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 18): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 19): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 20): <gurobi.QConstr Not Yet Added>,
 ((1, 1), 21): <gurobi.

#### (3.2.6) definicion de variable "z" (para dias con clase por profesor)

$$ z_{pd} = OR_h{y_{p(dh)}} $$

    para todo p, d

In [57]:
model.addConstrs(z_dict[p,d].variable ==
                gp.or_(y_dict[p,(d,h)].variable for h in horarios_ids)
                for p in profesores_ids for d in dias_ids)

{(0, 1): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 2): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 3): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 4): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 5): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 1): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 2): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 3): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 4): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 5): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 1): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 2): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 3): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 4): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 5): <gurobi.GenConstr *Awaiting Model Update*>,
 (3, 1): <gurobi.GenConstr *Awaiting Model Update*>,
 (3, 2): <gurobi.GenConstr *Awaiting Model Update*>,
 (3, 3): <gurobi.GenConstr *Awaiting Model Update*>,
 (3, 4): <gurobi.GenConstr *Awaiting Model Upd

Posible linealizacion:

$$ z_{pd} \leq \sum_h {y_{pdh}} \leq M \times z_{pd} $$

    para todo m, d


In [58]:
# for p in profesores:

#     model.addConstrs(gp.quicksum(y_dict[p.id, (d,h)].variable for h in horarios_ids)
#                      <= 10 * z_dict[p.id,d].variable for d in dias_ids)
    
#     model.addConstrs(gp.quicksum(y_dict[p.id, (d,h)].variable for h in horarios_ids)
#                      >= z_dict[p.id,d].variable for d in dias_ids)

#### (3.2.7) limitar la cantidad de horas diarias por profesor (NO VA)


$$ \sum_{m,h} {w_{mp}·u_{mdh}} \leq {HD}_p $$

    para todo p, d


In [59]:
# FALTA CHEQUEAR

# for p in profesores:
#     if not p.horas_max is None:
#         # mats = [search_materias_by_nombre(materias, m).id for m in p.materias()]
#         model.addConstrs(gp.quicksum(
#             w_dict[m, p.id].variable * u_dict[m,(d,h)].variable for m in materias_ids for h in horarios_ids
#             ) <= p.horas_max for d in dias_ids)

#### (3.2.8) limitar dias con clase por docente (NO VA)


$$ \sum_{m,d} {w_{mp}·v_{md}} \leq {Dmax}_p $$
$$ \sum_{d} {z_{pd}} \leq {Dmax}_p $$

    para todo p


In [60]:
# MAL IMPLEMENTADA

# for p in profesores:
#     if not p.cantidad_dias is None:
#         # mats = [search_materias_by_nombre(materias, m).id for m in p.materias()]
#         model.addConstr(gp.quicksum(
#             w_dict[m, p.id].variable * v_dict[m,d].variable for m in materias_ids for d in dias_ids
#             ) <= p.cantidad_dias)
        

In [61]:
# p = search_profesor_by_nombre(profesores, "jf")
# print(gp.quicksum(
#             w_dict[m, p.id].variable * v_dict[m,d].variable for m in materias_ids for d in dias_ids
#             ).getValue())

### (3.3) Restricciones externas

#### (3.3.1) cantidad de salones

$$ \sum_{m} {u_{mb}} \leq K $$

    para todo b

In [62]:
K = 13      # 13 salones en este caso

model.addConstrs(gp.quicksum(u_dict[m, b].variable for m in materias_ids) <= K for b in bloques_horario_ids)

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 6): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model Update*>,
 (2, 8): <gurobi.Constr *Awaiting Model Update*>,
 (2, 9): <gurobi.Constr *Awaiting Model Update*>,
 (2, 10): <gurobi.Constr *Awaiting Model Update*>

#### (3.3.2) fijar horas manualmente

In [63]:
# def fijar_ultimo_bloque(model, materia, dias_ids, horarios_ids):
#     model.addConstr(gp.quicksum(u_dict[materia.id, (d,h)].variable for d in dias_ids for h in horarios_ids[0:-1]) == 0)

# fijar practico de probabilidad al ultimo bloque
for m in search_materias_by_nombre(materias, "ProbP"): # fijar a ultimo bloque 
    if m.carga_horaria == 1:
        model.addConstr(gp.quicksum(u_dict[m.id, (d,h)].variable for d in dias_ids for h in horarios_ids_turno(horarios, "m")[0:-1]) == 0)


In [64]:

# fijar CreInv a los ultimos dos bloques
for m in search_materias_by_nombre(materias, "CreInv"):
    if m.carga_horaria == 2:
        model.addConstr(gp.quicksum(u_dict[m.id, (d,h)].variable for d in dias_ids for h in horarios_ids_turno(horarios, "m")[0:-2]) == 0)


In [65]:
# coincidir exactamente un bloque de dos horas de CyC
cyc = []
for m in search_materias_by_nombre(materias, "CyC"):
    cyc.append(m.id)

model.addConstr(gp.quicksum(u_dict[cyc[0], b].variable * u_dict[cyc[1], b].variable for b in bloques_horario_ids) == 2)



<gurobi.QConstr Not Yet Added>

In [66]:
# no coincidir dias de MecF
# FALTA CHEQUEAR

mecf = []
for m in search_materias_by_nombre(materias, "MecF"):
    mecf.append(m.id)

model.addConstr(gp.quicksum(v_dict[mecf[0], d].variable * v_dict[mecf[1], d].variable for d in dias_ids) == 0)

<gurobi.QConstr Not Yet Added>

In [67]:
# no dias consecutivos y no coincidir dia para ADA

ada1 = []
for m in materias_grupo(lista_grupos(grupos, ["2INF1"])[0], materias):
    if m.nombre == "ADA":
        ada1.append(m.id)

model.addConstr(gp.quicksum(v_dict[ada1[0],d].variable * v_dict[ada1[1],d+1].variable for d in dias_ids[0:-1]) == 0)
model.addConstr(gp.quicksum(v_dict[ada1[1],d].variable * v_dict[ada1[0],d+1].variable for d in dias_ids[0:-1]) == 0)
model.addConstr(gp.quicksum(v_dict[ada1[0],d].variable * v_dict[ada1[1],d].variable for d in dias_ids[0:-1]) == 0)

ada2 = []
for m in materias_grupo(lista_grupos(grupos, ["2INF2"])[0], materias):    
    if m.nombre == "ADA":
        ada2.append(m.id)

model.addConstr(gp.quicksum(v_dict[ada2[0],d].variable * v_dict[ada2[1],d+1].variable for d in dias_ids[0:-1]) == 0)
model.addConstr(gp.quicksum(v_dict[ada2[1],d].variable * v_dict[ada2[0],d+1].variable for d in dias_ids[0:-1]) == 0)
model.addConstr(gp.quicksum(v_dict[ada2[0],d].variable * v_dict[ada2[1],d].variable for d in dias_ids[0:-1]) == 0)


<gurobi.QConstr Not Yet Added>

In [68]:
# no dias consecutivos y no coincidir dia para IngSW2 grupo 3INF2
isw = []
for m in materias_grupo(lista_grupos(grupos, ["3INF2"])[0], materias):    
    if m.nombre == "IngSW2":
        isw.append(m.id)

model.addConstr(gp.quicksum(v_dict[isw[0],d].variable * v_dict[isw[1],d+1].variable for d in dias_ids[0:-1]) == 0)
model.addConstr(gp.quicksum(v_dict[isw[1],d].variable * v_dict[isw[0],d+1].variable for d in dias_ids[0:-1]) == 0)
model.addConstr(gp.quicksum(v_dict[isw[0],d].variable * v_dict[isw[1],d].variable for d in dias_ids[0:-1]) == 0)



<gurobi.QConstr Not Yet Added>

In [69]:
# mp: limitar a 4 horas por dia

mp = search_profesor_by_nombre(profesores, "mp")
horas_maximo = 4

model.addConstrs(gp.quicksum(y_dict[mp.id,(d,h)].variable for h in horarios_ids)
                <= horas_maximo for d in dias_ids)



{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

#### (3.3.3) practico despues del teorico

$$ \max_b\{(-b)*u_{mb}\} > \max_b\{(-b)*u_{m'b}\} $$

    para todo par (teorico, practico) =  (m, m')


In [70]:
teo = []    # array con IDS de materias de teorico
prac = []    # array con IDS de materias de practico
vars1erHora = {}
scaled_u_dict = {}

for nombre in ["AM2", "ALN", "AM1", "AL", "Prob", "ProgAv", "ProgAv"]:
    # se carga ProgAv dos veces a proposito
    for m in search_materias_by_nombre(materias, nombre):
        teo.append(m.id)
        vars1erHora[m.id] = model.addVar(vtype=GRB.INTEGER, name=str(m)+"_1er_hora")

for nombre in ["AM2p", "ALNp", "AM1p", "ALp", "ProbP", "ProgAvP"]:
    for m in search_materias_by_nombre(materias, nombre):
        prac.append(m.id)
        vars1erHora[m.id] = model.addVar(vtype=GRB.INTEGER, name=str(m)+"_1er_hora")

L = len(bloques_horario_ids)

for m in teo + prac:
    for b in range(0, L):
        scaled_u_dict[m, bloques_horario_ids[b]] = model.addVar(vtype=GRB.INTEGER, name=str(m)+"scaled_u")
        model.addConstr(scaled_u_dict[m, bloques_horario_ids[b]] == (L - b) * u_dict[m, bloques_horario_ids[b]].variable)

for m_id in vars1erHora:
    print(m_id, vars1erHora[m_id])
    model.addConstr(vars1erHora[m_id] ==
                    # gp.max_( (L - b) * u_dict[m_id, bloques_horario_ids[b]].variable
                    gp.max_( scaled_u_dict[m_id, bloques_horario_ids[b]]
                            for b in range(0, L)) )

if len(teo) == len(prac):
    model.addConstrs(vars1erHora[teo[m]] >= vars1erHora[prac[m]] for m in range(0, len(teo)))


0 <gurobi.Var *Awaiting Model Update*>
1 <gurobi.Var *Awaiting Model Update*>
2 <gurobi.Var *Awaiting Model Update*>
6 <gurobi.Var *Awaiting Model Update*>
7 <gurobi.Var *Awaiting Model Update*>
8 <gurobi.Var *Awaiting Model Update*>
21 <gurobi.Var *Awaiting Model Update*>
22 <gurobi.Var *Awaiting Model Update*>
27 <gurobi.Var *Awaiting Model Update*>
30 <gurobi.Var *Awaiting Model Update*>
31 <gurobi.Var *Awaiting Model Update*>
32 <gurobi.Var *Awaiting Model Update*>
67 <gurobi.Var *Awaiting Model Update*>
3 <gurobi.Var *Awaiting Model Update*>
4 <gurobi.Var *Awaiting Model Update*>
5 <gurobi.Var *Awaiting Model Update*>
9 <gurobi.Var *Awaiting Model Update*>
10 <gurobi.Var *Awaiting Model Update*>
11 <gurobi.Var *Awaiting Model Update*>
23 <gurobi.Var *Awaiting Model Update*>
24 <gurobi.Var *Awaiting Model Update*>
28 <gurobi.Var *Awaiting Model Update*>
33 <gurobi.Var *Awaiting Model Update*>
34 <gurobi.Var *Awaiting Model Update*>
35 <gurobi.Var *Awaiting Model Update*>
68 <gurobi

### (3.4) Grupos

#### (3.4.1) definicion de variable "x"

$$ x_{gb} = \sum_{m \in M_g} {u_{mb}} $$



In [71]:
for g in grupos:
    gr_mats_ids = materias_grupo_ids(g, materias)
    model.addConstrs(x_dict[g.id, b].variable == gp.quicksum(u_dict[m, b].variable for m in gr_mats_ids) for b in bloques_horario_ids)


#### (3.4.2) evitar horas puente por grupo

$$ \sum_h {x_{gdh}} - \sum_h {x_{gd(h)} · x_{gd(h+1)}} = 1 $$
    para todo g, d

In [72]:
grupos_reales = []
for g in grupos:
    if not g.anio is None:
        grupos_reales.append(g)

for g in grupos_reales:
    model.addConstrs(gp.quicksum(x_dict[(g.id,(d,h))].variable for h in horarios_ids)
                 - gp.quicksum(x_dict[(g.id,(d,h))].variable * x_dict[(g.id,(d,h+1))].variable for h in horarios_ids[0:-1])
                 == 1 for d in dias_ids)

## (4) Funcion Objetivo

In [73]:
# Set objective
OBJ = gp.QuadExpr()

### (4.1) Prioridad horaria de los docentes

$$ MIN: \sum_{m,b,p} {A_{pb}·w_{mp}·u_{mb}} $$


In [74]:
OBJ1 = gp.QuadExpr()

count = 0
for m in materias:
    for p in profesores:
        for pr in p.prioridades:
            b = pr.bloque_horario
            A = pr.value
            OBJ1 += A * w_dict[m.id, p.id].variable * u_dict[m.id, b.id()].variable
            count += 1

OBJ += OBJ1

print(count)


220000


LINEALIZAR: SUSTITUIR CON VARIABLES y_pb

$$ MIN: \sum_{p,b} {A_{pb}·y_{pb}} $$

In [75]:
# OBJ1 = gp.QuadExpr()

# count = 0
# for p in profesores:
#     for pr in p.prioridades:
#         b = pr.bloque_horario
#         A = pr.value
#         OBJ1 += A * y_dict[p.id, b.id()].variable
#         count += 1

# OBJ += OBJ1

# print(count)

### (4.2) Minimizar dias con clase por profesor
$$ MIN: \sum_{p,d} {OR_h{\sum_m{u_{mdh}·w_{mp}}}} $$
$$ MIN: \sum_{p,d} {z_{pd}} $$

In [76]:
OBJ2 = gp.LinExpr()
DP = {}

for p in profesores:
    
    D_p = gp.quicksum(z_dict[p.id,d].variable for d in dias_ids)
    DP[p.id] = D_p

    # filtrar si el profesor prefiere minimizar o no
    if p.minimizar_dias:
        OBJ2 += D_p

P = 5

OBJ += P*OBJ2


### (4.3) Minimizar horas puente por grupo (alternativa a la restriccion, NO VA)

$$ MIN: -\sum_{g,d,h} {x_{gd(h)}·x_{gd(h+1)}} $$

In [77]:
# OBJ3 = gp.QuadExpr()

# for g in grupos_reales:
#     for d in dias_ids:
#         OBJ3 += - gp.quicksum(x_dict[(g.id,(d,h))].variable * x_dict[(g.id,(d,h+1))].variable for h in horarios_ids[0:-1])
    
# # OBJ += OBJ3


## Resolver

In [78]:
model.setObjective(OBJ, GRB.MINIMIZE)

In [79]:
# for p in profesores:
#     print(str(p), p.lista_materias)

In [80]:
model.setParam("TimeLimit", 60*60*2)
model.optimize()

Set parameter TimeLimit to value 7200
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 10810 rows, 14053 columns and 38345 nonzeros
Model fingerprint: 0x9defbe52
Model has 103200 quadratic objective terms
Model has 8989 quadratic constraints
Model has 302 general constraints
Variable types: 0 continuous, 14053 integer (12625 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  QMatrix range    [1e+00, 2e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [5e+00, 5e+00]
  QObjective range [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
  QRHS range       [1e+00, 2e+00]
Presolve removed 8196 rows and 11130 columns
Presolve time: 0.37s
Presolved: 9678 rows, 5236 columns, 30627 nonzeros
Variable types: 0 continuous, 5236 integer (5084 binary)


In [81]:

# if model.Status == GRB.INFEASIBLE:
#     model.computeIIS()
#     model.write('iismodel.ilp')

#     # Print out the IIS constraints and variables
#     print('\nThe following constraints and variables are in the IIS:')
#     for c in model.getConstrs():
#         if c.IISConstr: print(f'\t{c.constrname}: {model.getRow(c)} {c.Sense} {c.RHS}')

#     for v in model.getVars():
#         if v.IISLB: print(f'\t{v.varname} ≥ {v.LB}')
#         if v.IISUB: print(f'\t{v.varname} ≤ {v.UB}')

### Imprimir horarios

In [113]:
if not model.Status == GRB.INFEASIBLE:
    print('Obj: %g' % model.ObjVal)
    print('OBJ1: %g' % OBJ1.getValue())
    print('OBJ2: %g' % OBJ2.getValue())
    print('=============================================================')
    print_timetable(dias, horarios, u_dict, w_dict, grupos, anios)

    copy_variables_excel(u_dict, w_dict)


Obj: 476
OBJ1: 396
OBJ2: 16

 -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
Año:  1

 Grupo:  11
			lun		mar		mie		jue		vie
··········································································································
8:00-8:50		---		DBD1		Antrop		DBD1		Antrop
8:50-9:40		AM2		DBD1		Antrop		DBD1		Antrop
9:50-10:40		AM2		DBD1		Fisica		AM2		Fisica
10:40-11:30		AM2		ALN		Fisica		AM2		Fisica
11:40-12:30		AM2p		ALN		Fisica		ALN		ALNp
12:30-13:20		AM2p		ALN		---		ALN		ALNp

 Grupo:  12
			lun		mar		mie		jue		vie
··········································································································
8:00-8:50		ALN		Antrop		ALN		---		ALNp
8:50-9:40		ALN		Antrop		ALN		Antrop		ALNp
9:50-10:40		DBD1		Fisica		DBD1		Antrop		Fisica
10:40-11:30		DBD1		Fisica		DBD1		AM2		Fisica
11:40-12:30		DBD1		AM2		AM2p		AM2		Fisica
12:30-13:20		---		AM2		AM2p		AM2		---

 Grupo:  13
			lun		mar		mie		jue		vie
·····················

In [83]:
# HORAS PUENTE

# HP = gp.QuadExpr()
# for g in grupos_reales:
#     HPg = gp.QuadExpr()
#     for d in dias_ids:
#         HPg += gp.quicksum(x_dict[(g.id,(d,h))].variable for h in horarios_ids)
#         HPg += - gp.quicksum(x_dict[(g.id,(d,h))].variable * x_dict[(g.id,(d,h+1))].variable for h in horarios_ids[0:-1])
#         HPg += - 1
#     # print("horas puente grupo", str(g), ": ", HPg.getValue())
#     HP += HPg

# print("Total de horas puente: ", HP.getValue())

In [84]:
# DIAS CLASE POR PROFESOR

# print("DIAS CON CLASE")
# for p in profesores:
#     print(str(p), ": ", round(DP[p.id].getValue()))

In [85]:
# print_prof_timetable(dias, horarios, u_dict, w_dict, profesores, search_materias_by_nombre(materias, "IngSW2"))
# print_prof_timetable(dias, horarios, u_dict, w_dict, profesores, search_materias_by_nombre(materias, "ADA"))

# print_prof_timetable_excel(dias, horarios, u_dict, w_dict, profesores, materias)

In [112]:
print_prof_timetable(dias, horarios, u_dict, w_dict, [search_profesor_by_nombre(profesores, "jk")], materias)
# print_prioridades(dias, horarios, [search_profesor_by_nombre(profesores, "jk")])
# print_prof_timetable(dias, horarios, u_dict, w_dict, [search_profesor_by_nombre(profesores, "jpa")], materias)
# print_prof_timetable(dias, horarios, u_dict, w_dict, [search_profesor_by_nombre(profesores, "jb")], materias)


 Profesor:  jk
		lun	mar	mie	jue	vie
·····················································
8:00-8:50	---	---	---	---	---
8:50-9:40	---	---	---	---	---
9:50-10:40	---	---	---	---	---
10:40-11:30	Prob	---	Prob	---	---
11:40-12:30	Prob	---	Prob	---	---
12:30-13:20	---	---	---	---	---
14:00-14:50	AL	---	ALp	---	---
14:50-15:40	AL	---	ALp	---	---
15:50-16:40	AL	---	AL	---	---
16:40-17:30	---	---	AL	---	---


In [87]:
# print_prioridades(dias, horarios, [search_profesor_by_nombre(profesores, "gr")])
# print_prioridades(dias, horarios, [search_profesor_by_nombre(profesores, "ig")])

In [88]:
# imprimir calendario para profesores
# if not model.Status == GRB.INFEASIBLE:
    # print_prof_timetable(dias, horarios, u_dict, w_dict, [search_profesor_by_nombre(profesores, "gc")], materias)
    # print_prof_timetable(dias, horarios, u_dict, w_dict, profesores, search_materias_by_nombre(materias,"Prob"))
    # print_prof_timetable(dias, horarios, u_dict, w_dict, profesores, materias)

In [89]:
# IMPRIMIR PARA COPIAR EN HOJA DE EXCEL
print_timetable_excel(dias, horarios, u_dict, w_dict, grupos, [anios[0]])


 -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
Año:  1

 Grupo:  11
	lun	mar	mie	jue	vie
8:00-8:50	---	DBD1 ['dl']	Antrop []	DBD1 ['dl']	Antrop []
8:50-9:40	AM2 ['jd']	DBD1 ['dl']	Antrop []	DBD1 ['dl']	Antrop []
9:50-10:40	AM2 ['jd']	DBD1 ['dl']	Fisica ['jpf']	AM2 ['jd']	Fisica ['jpf']
10:40-11:30	AM2 ['jd']	ALN ['lg']	Fisica ['jpf']	AM2 ['jd']	Fisica ['jpf']
11:40-12:30	AM2p ['mp']	ALN ['lg']	Fisica ['jpf']	ALN ['lg']	ALNp ['jf']
12:30-13:20	AM2p ['mp']	ALN ['lg']	---	ALN ['lg']	ALNp ['jf']

 Grupo:  12
	lun	mar	mie	jue	vie
8:00-8:50	ALN ['ap']	Antrop []	ALN ['ap']	---	ALNp ['jf']
8:50-9:40	ALN ['ap']	Antrop []	ALN ['ap']	Antrop []	ALNp ['jf']
9:50-10:40	DBD1 ['ac']	Fisica ['gb']	DBD1 ['ac']	Antrop []	Fisica ['gb']
10:40-11:30	DBD1 ['ac']	Fisica ['gb']	DBD1 ['ac']	AM2 ['am']	Fisica ['gb']
11:40-12:30	DBD1 ['ac']	AM2 ['am']	AM2p ['mp']	AM2 ['am']	Fisica ['gb']
12:30-13:20	---	AM2 ['am']	AM2p ['mp']	AM2 ['am']	---

 Grupo:  13
	lun

In [90]:
# print_prof_timetable_excel(dias, horarios, u_dict, w_dict, profesores, materias)

In [91]:
# for m in materias:
#     print(str(m), ":", [str(g) for g in m.grupos])
    # print(m, [str(p) for p in m.profesores])

In [92]:
# for p in profesores:
#     print(str(p), [str(m) for m in materias_profesor(p, materias)])

### Promedios de prioridad

In [93]:
# CARGA_HORARIA_TOTAL = gp.quicksum(m.carga_horaria for m in materias)
CARGA_HORARIA_TOTAL = gp.quicksum(u_dict[m, b].variable * w_dict[m, p].variable for p in profesores_ids for m in materias_ids for b in bloques_horario_ids)

if not model.Status == GRB.INFEASIBLE:
    print(OBJ1.getValue())
    print("Promedio general de prioridad horaria: ", OBJ1.getValue()/CARGA_HORARIA_TOTAL.getValue())
          

# print("Casos de materias con dias consecutivos: ", int(OBJ2.getValue()/10))


396.0
Promedio general de prioridad horaria:  1.32


In [94]:
# promedio por profesor:
for p in profesores:

    OBJ_p = gp.QuadExpr()
    # for m in materias:
    #     for pr in p.prioridades:
    #             b = pr.bloque_horario
    #             A = pr.value
    #             OBJ_p += A * w_dict[m.id, p.id].variable * u_dict[m.id, b.id()].variable

    for pr in p.prioridades:
        b = pr.bloque_horario
        A = pr.value
        OBJ_p += A * y_dict[p.id, b.id()].variable
    
    # CARGA_HORARIA_p = gp.quicksum(u_dict[m, b].variable * w_dict[m, p.id].variable for m in materias_ids for b in bloques_horario_ids).getValue()
    CARGA_HORARIA_p = round(gp.quicksum(y_dict[p.id, b].variable for b in bloques_horario_ids).getValue())

    if CARGA_HORARIA_p != 0:
        print(str(p), "\t", CARGA_HORARIA_p, "\t", round(round(OBJ_p.getValue())/CARGA_HORARIA_p*1000))
    else:
        print(str(p), "sin carga horaria")


jd 	 9 	 1000
gc 	 10 	 1000
am 	 10 	 1000
mp 	 10 	 1100
lg 	 5 	 1000
svv 	 5 	 1000
ap 	 4 	 1000
jf 	 6 	 1000
jpf 	 5 	 1000
gb 	 11 	 1182
gs 	 5 	 1000
dl 	 5 	 1000
sgp 	 5 	 1200
ac 	 5 	 1200
ps 	 4 	 1000
jk 	 11 	 1909
ds 	 5 	 3000
rl 	 4 	 1000
nn 	 4 	 1000
guc 	 10 	 1300
mjc 	 10 	 1300
sb 	 10 	 1300
ma 	 6 	 1000
dc 	 8 	 1250
jc 	 10 	 2500
cb 	 5 	 1800
as 	 5 	 1000
ms 	 3 	 1000
if 	 4 	 1500
bt 	 5 	 1400
nm 	 4 	 2000
mg 	 2 	 1000
mb 	 2 	 1000
jcc 	 2 	 1000
le 	 2 	 1000
jp 	 5 	 1600
jg 	 4 	 1000
jpe 	 8 	 1250
em 	 2 	 3000
fc 	 3 	 1000
gr 	 4 	 1500
ig 	 4 	 2500
dv 	 4 	 1500
jb 	 2 	 2000
jds 	 2 	 2000
mpi 	 4 	 1000
sp 	 4 	 1000
jpa 	 5 	 1400
mpr 	 5 	 1400
el 	 4 	 1500
ep 	 5 	 1200
dj 	 6 	 1167
rb 	 6 	 1333
mr 	 6 	 1000
sv 	 6 	 1000


In [95]:
# distribucion de prioridades

promedios_prioridad = {}
valores_prioridad = {}
for i in range(1,4):
    promedios_prioridad[i] = 0
    valores_prioridad[i] = 0

for p in profesores:
    for pr in p.prioridades:
        b = pr.bloque_horario
        A = pr.value
        if A == 0:
            continue
        valores_prioridad[A] += 1
        # for m in materias:
        #     promedios_prioridad[A] += round(w_dict[m.id, p.id].variable.X * u_dict[m.id, b.id()].variable.X)
        promedios_prioridad[A] += round(y_dict[p.id, b.id()].variable.X)

print(valores_prioridad)
total = sum([valores_prioridad[i] for i in valores_prioridad])
for i in valores_prioridad:
    print(i, ": ", str(round(valores_prioridad[i]/total*100))+"%")

print(promedios_prioridad)
total = sum([promedios_prioridad[i] for i in promedios_prioridad])
for i in promedios_prioridad:
    print(i, ": ", str(round(promedios_prioridad[i]/total*100))+"%")
    
    # CARGA_HORARIA_p = gp.quicksum(u_dict[m, b].variable * w_dict[m, p.id].variable for m in materias_ids for b in bloques_horario_ids).getValue()

    # if CARGA_HORARIA_p != 0:
    #     print(str(p), ": ", OBJ_p.getValue()/CARGA_HORARIA_p)
    # else:
    #     print(str(p), "sin carga horaria")

{1: 699, 2: 288, 3: 303}
1 :  54%
2 :  22%
3 :  23%
{1: 230, 2: 44, 3: 26}
1 :  77%
2 :  15%
3 :  9%


### Ocupacion de salones

In [96]:
# nivel de ocupacion
niveles = {}

for b_id in bloques_horario:
    salones_ocupados = int(gp.quicksum(u_dict[m, b_id].variable for m in materias_ids).getValue())
    nivel = int(salones_ocupados/K*100)
    # print(str(bloques_horario[b_id]), ": ", salones_ocupados,",", str(nivel) + "%")
    niveles[b_id] = nivel

print_timetable_salones(dias, horarios, niveles)




		lun	mar	mie	jue	vie
··········································································································
8:00-8:50	46%	76%	61%	69%	69%
8:50-9:40	84%	84%	61%	92%	69%
9:50-10:40	84%	76%	84%	100%	61%
10:40-11:30	76%	76%	84%	76%	61%
11:40-12:30	69%	46%	69%	69%	53%
12:30-13:20	53%	46%	23%	46%	46%
14:00-14:50	7%	15%	7%	15%	15%
14:50-15:40	7%	15%	7%	15%	15%
15:50-16:40	7%	7%	7%	15%	15%
16:40-17:30	0%	7%	7%	0%	15%


In [97]:
# for u_i in u_dict:
#     u = u_dict[u_i]
#     print(str(u) + " = ", u.variable.X)

# for v_i in v_dict:
#     v = v_dict[v_i]
#     print(str(v) + " = ", v.variable.X)

# for w_i in w_dict:
#     w = w_dict[w_i]
#     print(str(w) + " = ", w.variable.X)


In [98]:
model.write('timetable_opt.lp')
model.write('timetable_opt.mps')

# model.display()